# JAMy_cest_pas_sorcier Project
### Alexandre Flachs, Marie Giot & Jeanne Szpirer

#### Participation to "Flu Shot Learning: Predict H1N1 and Seasonal Flu Vaccines" competition

First we need to charge all the data (training set features, test set features and training set labels).

In [ ]:
# Training set features
training_set_features <- read.csv("training_set_features.csv")
dim(training_set_features)

In [ ]:
# Test set features
test_set_features <- read.csv("test_set_features.csv")
dim(test_set_features)

In [ ]:
# Training set labels
training_set_labels <- read.csv("training_set_labels.csv")
dim(training_set_labels)

We can see that the training features set and the training labels set has the same amount of lines, this is a first good sign because it means that we have an "answer" for every training line.